Альтернатива 1-ой задаче. Код писался в PyCharm (под средой анаконды)

In [15]:
# Импортируем нужные библиотеки

import requests
from bs4 import BeautifulSoup
import pandas as pd

"""
Создаем константы:
    URL - адреса страниц, с которых будем парсить.
    HEADERS - словарь заголовков, для имитации живого пользователя
    HOST - для дальнейшей контатенации ссылок (парсится только последняя часть без хоста, 
                                               а нам нужно кликабельная)
"""

URL = 'https://auto.ria.com/newauto/marka-jeep/'

URL1 = 'https://auto.ria.com/uk/newauto/marka-jaguar/'

URL2 = 'https://auto.ria.com/uk/newauto/marka-alfa-romeo/'

HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}  
HOST = 'https://auto.ria.com'

"""
Разбиваем все по функциям:
get_html - юзает библиотеку requests и забирает инфу по передаваемому в параметрах URL. 
Второй параметр - параметры запроса (нужно будет для обработки пагинации, передавать ?page= 'n').
"""

def get_html(url, params=None):

    req = requests.get(url, headers=HEADERS, params=params)
    return req

"""
get_pages_count - как раз функция для сбора количества страниц (идем в нижний блок с кнопками страниц,
собираем все и возвращаем из функции только последнюю - int(pagination[-1].get_text())). Если 
пагинации нет (список машин короткий), то возвращаем 1. Принимает на вход html, передаваемый 
из get_html
"""

def get_pages_count(html):

    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find_all('span', class_='page-item mhide')

    if pagination:
        return int(pagination[-1].get_text())
    else:
        return 1

"""
get_content - принимает на вход html, передаваемый из get_html. Отвечает за непосредственный парсинг 
страницы. NB - рассчитано только для страниц с новыми авто. На других будет возвращать пустоту - 
верстка уже другая.
"""

"""
Проходимся циклом по каждой карточке и поочередно собираем нужные нам данные из конкретных тегов. 
Предварительно приходится проводить проверку на наличие информации по ценам - ее зачастую нет. 
Что приводит к ошибкам попытки обратиться к None с помощью методов BeautifulSoup. Если информации 
нет - подставляем предзаписанное сообщение. В конце возвращаем список машин с текущей страницы
"""

def get_content(html):
    # Ищем нужные нам теги, отображающие карточки товаров (в карточках вся нужная нам инфа)
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='proposition')

    # Пустой список для собранных авто
    cars = []
    
    for item in items:
        grvn_price = item.find('span', class_="grey size13")
        usd_price = item.find('div', class_='proposition_price').find_next('span', class_="green bold size18 tooltip-price")

        if grvn_price:
            grvn_price = grvn_price.get_text(strip=True)
        else:
            grvn_price = 'Цена в гривнах неизвестна'

        if usd_price:
            usd_price = usd_price.get_text(strip=True)
        else:
            usd_price = 'Цена в долларах неизвестна'

        cars.append({
            'title': item.find('div', class_="proposition_title").get_text(strip=True),
            'link': HOST + item.find('a').get('href'),
            'usd_price': usd_price,
            'grvn_price': grvn_price,
            'city': item.find('div', class_='proposition_region size13').find_next('strong').get_text(strip=True)
        })
    return cars

"""
Функция 'main' - точка запуска парсинга. Вызывает get_html, передавая нужный URL, далее проверка 
на статус-код ответа, если 200, то вызывая функция для подсчета страниц - get_pages_count(html.text). 
И уже проходясь циклом по всем полученным страницам, подставляя с помощью передавемого в запросе 
параметра 'page' текущий номер, запускает функцию-парсер. В конце каждой итерации расширяем наш 
пустой список новым, собранным с новой страницы. В конце выводим весь список целиком и общее число 
собранных авто.
"""

def parse(URL):

    html = get_html(URL)

    if html.status_code == 200:
        cars = []
        pages_count = get_pages_count(html.text)
        for page in range(1, pages_count+1):
            print(f'Парсинг страницы {page} из {pages_count}')
            html = get_html(URL, params={'page': page})
            cars.extend(get_content(html.text))
        print()
        print(f'Собрана информация о {len(cars)} авто')
        print()
        df = pd.DataFrame(cars)
        return df
    else:
        print('Error')

In [17]:
# Запуск парсинга

parse(URL1)

# Единственная проблема возникает с парсингом цен в долларах. Прямо в процессе была изменена верстка,
# она там гуляет от раздела к разделу, и вытащить этот ценник теперь не удается. Хотя ценник в гривнах,
# который лежит в том же div, спокойно парсится. Достучаться к долларам не получается ни обращаясь 
# напрямую к нужному span, ни через родительский div.

# И еще такой момент смущает. У меня здесь список словарей. Просто создать df получается, но при попытке
# создать его, сразу передав названия столбцов, получаю таблицу NaN-ов.


Парсинг страницы 1 из 4
Парсинг страницы 2 из 4
Парсинг страницы 3 из 4
Парсинг страницы 4 из 4

Собрана информация о 66 авто



,title,link,usd_price,grvn_price,city
0,Jaguar F-Pace 2020R-Sport 2.0D AT (180 к.с),https://auto.ria.com/newauto/auto-jaguar-f-pac...,Цена в долларах неизвестна,1 450 130 грн,Київ
1,Jaguar I-Pace 2020HSE EV400 90kWh AWD,https://auto.ria.com/newauto/auto-jaguar-i-pac...,Цена в долларах неизвестна,2 760 706 грн,Одеса
2,Jaguar I-Pace 2020HSE EV400 90kWh AWD,https://auto.ria.com/newauto/auto-jaguar-i-pac...,Цена в долларах неизвестна,2 719 079 грн,Одеса
3,Jaguar F-Type 2020First Edition P300 2.0 Si4 A...,https://auto.ria.com/newauto/auto-jaguar-f-typ...,Цена в долларах неизвестна,3 055 606 грн,Чубинське
4,Jaguar F-Pace 2019Prestige 2.0D AT (180 к.с) AWD,https://auto.ria.com/newauto/auto-jaguar-f-pac...,Цена в долларах неизвестна,1 569 552 грн,Одеса
...,...,...,...,...,...
61,Jaguar I-Pace 2020S EV400 90kWh AWD,https://auto.ria.com/newauto/auto-jaguar-i-pac...,Цена в долларах неизвестна,2 273 840 грн,Дніпро (Дніпропетровськ)
62,Jaguar E-Pace 2020S 2.0 AT (200 к.с) AWD,https://auto.ria.com/newauto/auto-jaguar-e-pac...,Цена в долларах неизвестна,1 360 007 грн,Дніпро (Дніпропетровськ)
63,Jaguar F-Pace 2020R-Sport 2.0D AT (180 к.с),https://auto.ria.com/newauto/auto-jaguar-f-pac...,Цена в долларах неизвестна,1 739 091 грн,Дніпро (Дніпропетровськ)
64,Jaguar E-Pace 2020R-Dynamic S 2.0D AT (150 к.с...,https://auto.ria.com/newauto/auto-jaguar-e-pac...,Цена в долларах неизвестна,1 433 378 грн,Дніпро (Дніпропетровськ)


Альтернатива 1-ой задачи. Черновой вариант. Сайт оказался слишком топорно сделанным и упрощенным.

Задание 1.
Обязательная часть
Будем парсить страницу со свежими новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [78]:
"""
Первый черновой вариант альтернативы)
"""
import requests
from bs4 import BeautifulSoup

URL = 'https://ntinews.ru/news/khronika-rynkov-nti/technet/'

# получаем страницу с новостями
req = requests.get(URL)
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="technet" name="keywords"/>
<meta content="14 февраля в 10.45 начнется презентация образовательного интенсива, которая пройдет в Рабочей зоне СМИ Главного медиацентра в Сочи. На главной странице ntinews.ru будет вестись прямая трансляция." name="description"/>
<link data-template-style="true" href="/local/templates/2035ib/styles.css?1577977816829" rel="stylesheet" type="text/css"/>
<script type="text/javascript">var _ba = _ba || []; _ba.push(["aid", "1a24b0301d1cbfdfd62dc3559bb8469e"]); _ba.push(["host", "ntinews.ru"]); (function() {var ba = document.createElement("script"); ba.type = "text/javascript"; ba.async = true;ba.src = (document.location.protocol == "https:" ? "https://" : "http://") + "bitrix.info/ba.js";var s = document.getElementsByTagName("script")[0];s.parentNode.insertBefore(ba, s);})();</script>
<title>TechNet</title>
<meta content="IE=edge" http-e

In [48]:
# извлекаем даты
dates_text = []

dates = soup.find_all('div', class_='date')

for date in dates:
    date_txt = date.text
    dates_text.append(date_txt)

dates_text

['20.04.2020',
 '11.03.2020',
 '10.03.2020',
 '03.03.2020',
 '21.02.2020',
 '20.02.2020',
 '19.02.2020',
 '17.02.2020',
 '17.02.2020']

In [45]:
# заголовки новостей

titles_text = []

titles = soup.find_all('p', class_='title')

for title in titles:
    t_text = title.text
    titles_text.append(t_text)
    
titles_text

['Робот-диагност, дезинфектор «Скорпион» и медицинский чат-бот предлагаются как российские антивирусные продукты ',
 '«Промобот» представил в Нью-Йорке робота-няню',
 'Центр компетенций НТИ СПбПУ открыл курс «Технологии цифровой  промышленности»',
 'Открылся прием заявок на конкурс MANUNET для разработчиков новых производственных технологий',
 'В СПбПУ оценили разработку цифрового двойника перспективного авиадвигателя ТВ7-117СТ',
 'СПбПУ откроет Зеркальный инжиниринговый центр в Ижевске',
 'СПбПУ открывает пилотный набор слушателей на программу ЕМВА «Лидеры цифровой трансформации»',
 'Учителей Петербурга и Финляндии подготовят для обучения технологиям Индустрии 4.0',
 'В Красноярске завершился чемпионат России по робототехнике First Russia Robotics Championship',
 'Искусственный интеллект в медицине']

In [44]:
# ссылки
links = []

for title in titles:
    link = title.find('a')
    links.append(link.attrs.get('href'))
    
links


['/news/khronika-rynkov-nti/technet/robot-diagnost-dezinfektor-skorpion-i-meditsinskiy-chat-bot-predlagayutsya-kak-rossiyskie-antivirusn.html',
 '/news/khronika-rynkov-nti/technet/promobot-predstavil-v-nyu-yorke-robota-nyanyu.html',
 '/news/khronika-rynkov-nti/technet/tsentr-kompetentsiy-nti-spbpu-otkryl-kurs-tekhnologii-tsifrovoy-promyshlennosti.html',
 '/news/khronika-rynkov-nti/technet/s-3-marta-startoval-priem-zayavok-na-konkurs-manunet.html',
 '/news/khronika-rynkov-nti/technet/v-spbpu-otsenili-razrabotku-tsifrovykh-dvoynikov-perspektivnogo-aviadvigatelya-tv7-117st.html',
 '/news/khronika-rynkov-nti/technet/spbpu-otkroet-zerkalnyy-inzhiniringovyy-tsentr-v-izhevske.html',
 '/news/khronika-rynkov-nti/technet/spbpu-otkryvaet-pilotnyy-nabor-slushateley-v-programmu-emva-lidery-tsifrovoy-transformatsii.html',
 '/news/khronika-rynkov-nti/technet/uchiteley-peterburga-i-finlyandii-podgotovyat-dlya-obucheniya-tekhnologiyam-industrii-4-0.html',
 '/news/khronika-rynkov-nti/technet/v-krasnoyar

In [25]:
for i in range(len(titles)):
        print(posts[i].text, ':', posts[i].attrs.get('href'))


Робот-диагност, дезинфектор «Скорпион» и медицинский чат-бот предлагаются как российские антивирусные продукты  : None
«Промобот» представил в Нью-Йорке робота-няню : None
Центр компетенций НТИ СПбПУ открыл курс «Технологии цифровой  промышленности» : None
Открылся прием заявок на конкурс MANUNET для разработчиков новых производственных технологий : None
В СПбПУ оценили разработку цифрового двойника перспективного авиадвигателя ТВ7-117СТ : None
СПбПУ откроет Зеркальный инжиниринговый центр в Ижевске : None
СПбПУ открывает пилотный набор слушателей на программу ЕМВА «Лидеры цифровой трансформации» : None
Учителей Петербурга и Финляндии подготовят для обучения технологиям Индустрии 4.0 : None
В Красноярске завершился чемпионат России по робототехнике First Russia Robotics Championship : None
Искусственный интеллект в медицине : None


In [52]:
import pandas as pd

def news20_35():
    KEYWORDS = ['робот', 'робота', 'robot']
    URL = 'https://ntinews.ru/news/khronika-rynkov-nti/technet/'

    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    dates_text = []
    titles_text = []
    links = []

    dates = soup.find_all('div', class_='date')
    titles = soup.find_all('p', class_='title')

    for date in dates:
        date_txt = date.text
        dates_text.append(date_txt)

    for title in titles:
        t_text = title.text
        titles_text.append(t_text)

    for title in titles:
        link = title.find('a')
        links.append(link.attrs.get('href'))
        
    df = pd.DataFrame(list(zip(dates_text, titles_text, links)), 
                            columns = ['Date', 'Titles', 'Links'])
    return df


news20_35()
    

,Date,Titles,Links
0,20.04.2020,"Робот-диагност, дезинфектор «Скорпион» и медиц...",/news/khronika-rynkov-nti/technet/robot-diagno...
1,11.03.2020,«Промобот» представил в Нью-Йорке робота-няню,/news/khronika-rynkov-nti/technet/promobot-pre...
2,10.03.2020,Центр компетенций НТИ СПбПУ открыл курс «Техно...,/news/khronika-rynkov-nti/technet/tsentr-kompe...
3,03.03.2020,Открылся прием заявок на конкурс MANUNET для р...,/news/khronika-rynkov-nti/technet/s-3-marta-st...
4,21.02.2020,В СПбПУ оценили разработку цифрового двойника ...,/news/khronika-rynkov-nti/technet/v-spbpu-otse...
5,20.02.2020,СПбПУ откроет Зеркальный инжиниринговый центр ...,/news/khronika-rynkov-nti/technet/spbpu-otkroe...
6,19.02.2020,СПбПУ открывает пилотный набор слушателей на п...,/news/khronika-rynkov-nti/technet/spbpu-otkryv...
7,17.02.2020,Учителей Петербурга и Финляндии подготовят для...,/news/khronika-rynkov-nti/technet/uchiteley-pe...
8,17.02.2020,В Красноярске завершился чемпионат России по р...,/news/khronika-rynkov-nti/technet/v-krasnoyars...


In [76]:
urls_pages = []
pages = soup.find('div', class_='pagination')
links_for_pages = pages.find_all('li')

for li in links_for_pages:
    link_p = li.find('a')
    if link_p != None:
        urls_pages.append(link_p.attrs.get('href'))
        
urls_pages

['/news/khronika-rynkov-nti/technet/',
 '/news/khronika-rynkov-nti/technet/',
 '/news/khronika-rynkov-nti/technet/?PAGEN_1=3',
 '/news/khronika-rynkov-nti/technet/?PAGEN_1=4',
 '/news/khronika-rynkov-nti/technet/?PAGEN_1=5',
 '/news/khronika-rynkov-nti/technet/?PAGEN_1=3']

In [79]:
for url in urls_pages[2:]:
    req = requests.get('https://ntinews.ru' + url)
    soup_url = BeautifulSoup(req.text, 'html.parser')
    
soup_url

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="technet" name="keywords"/>
<meta content="14 февраля в 10.45 начнется презентация образовательного интенсива, которая пройдет в Рабочей зоне СМИ Главного медиацентра в Сочи. На главной странице ntinews.ru будет вестись прямая трансляция." name="description"/>
<link data-template-style="true" href="/local/templates/2035ib/styles.css?1577977816829" rel="stylesheet" type="text/css"/>
<script type="text/javascript">var _ba = _ba || []; _ba.push(["aid", "1a24b0301d1cbfdfd62dc3559bb8469e"]); _ba.push(["host", "ntinews.ru"]); (function() {var ba = document.createElement("script"); ba.type = "text/javascript"; ba.async = true;ba.src = (document.location.protocol == "https:" ? "https://" : "http://") + "bitrix.info/ba.js";var s = document.getElementsByTagName("script")[0];s.parentNode.insertBefore(ba, s);})();</script>
<title>TechNet</title>
<meta content="IE=edge" http-e

Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
